In [2]:
'''
Data is based on the transcation history on 2026-01-01
'''

'\nData is based on the transcation history on 2026-01-01\n'

In [4]:
import networkx as nx
import pandas as pd
import math
from plotnine import ggplot, aes, geom_bar, labs, theme_bw,geom_col, element_text, theme,geom_histogram, scale_x_log10,scale_y_log10 , facet_wrap, element_rect
from collections import Counter

# File paths for input and output data
inputs = r"C:\Users\Shenghui\Documents\GitHub\MSc_DV_CC\Task3\blockchair_bitcoin_inputs_20260101.tsv"
outputs = r"C:\Users\Shenghui\Documents\GitHub\MSc_DV_CC\Task3\blockchair_bitcoin_outputs_20260101.tsv"

In [5]:
# Load data
idf = pd.read_csv(inputs,sep="\t")
odf = pd.read_csv(outputs,sep="\t")

In [6]:
# Display first 5 rows of input data
idf.head()

,block_id,transaction_hash,index,time,value,value_usd,recipient,type,script_hex,is_from_coinbase,...,spending_block_id,spending_transaction_hash,spending_index,spending_time,spending_value_usd,spending_sequence,spending_signature_hex,spending_witness,lifespan,cdd
0,925940,1c8fddb2190947d73d8abf570b6acbc0f71c14ddfa2726...,0,2025-12-01 00:04:53,226510,204.4117,1FGiZB7K757EUixGcyeyME6Jp8qQZEiUUk,pubkeyhash,76a9149c8ab044348d826b9ae88d698d575a45a6e8fc69...,0,...,930341,ed447d4fac95217c5ca6a18948b8fd0ded3422b4ff4d9c...,0,2026-01-01 00:06:10,198.2325,4294967295,47304402206e8cb8c09e63ee1581f27a29296f35be5aa2...,NaN,2678477,0.070220
1,925940,edb174b0c2b5d50cd4204991fe98eae0653eec81c24ccc...,0,2025-12-01 00:04:53,161927,146.1294,1FGiZB7K757EUixGcyeyME6Jp8qQZEiUUk,pubkeyhash,76a9149c8ab044348d826b9ae88d698d575a45a6e8fc69...,0,...,930341,ed447d4fac95217c5ca6a18948b8fd0ded3422b4ff4d9c...,1,2026-01-01 00:06:10,141.7120,4294967295,4730440220095a6339ff14bb4e00f182df7ae915947c7e...,NaN,2678477,0.050199
2,925945,7d60e781c169bd564a848554ff62b6145fe4b81ae5783b...,0,2025-12-01 01:22:43,231716,209.1098,1GzgDCeDyeb4sKQZJZu5rUjAcgRvBBJiY7,pubkeyhash,76a914af7253e68afa9b6bde29f15829b27fd9c3f6dd21...,0,...,930341,ea9e624f3d62e5b9c3d685b6183d96ae9d263675e77440...,0,2026-01-01 00:06:10,202.7886,4294967295,473044022073c2312d24758ca5a592ea37d20883d27aa5...,NaN,2673807,0.071709
3,925943,125798cb5ea3af6e342561c2a670f3d70159b26b063549...,0,2025-12-01 00:42:53,152700,137.8026,1GzgDCeDyeb4sKQZJZu5rUjAcgRvBBJiY7,pubkeyhash,76a914af7253e68afa9b6bde29f15829b27fd9c3f6dd21...,0,...,930341,ea9e624f3d62e5b9c3d685b6183d96ae9d263675e77440...,1,2026-01-01 00:06:10,133.6369,4294967295,47304402206eeb5a1820a8c7305bab42d3e580e043ec15...,NaN,2676197,0.047298
4,925945,67cbe3c8c0a185fca6626cc1980a32912f7e58e8c03890...,0,2025-12-01 01:22:43,231716,209.1098,1GzgDCeDyeb4sKQZJZu5rUjAcgRvBBJiY7,pubkeyhash,76a914af7253e68afa9b6bde29f15829b27fd9c3f6dd21...,0,...,930341,809e1a85a322210d6e1fdf581c57d121e7afc3818ca176...,0,2026-01-01 00:06:10,202.7886,4294967295,47304402202260e21c338a3894b2c7ff19bb21edffb7bf...,NaN,2673807,0.071709


In [7]:
# Display data types of input dataframe
idf.dtypes

block_id                       int64
transaction_hash              object
index                          int64
time                          object
value                          int64
value_usd                    float64
recipient                     object
type                          object
script_hex                    object
is_from_coinbase               int64
is_spendable                   int64
spending_block_id              int64
spending_transaction_hash     object
spending_index                 int64
spending_time                 object
spending_value_usd           float64
spending_sequence              int64
spending_signature_hex        object
spending_witness              object
lifespan                       int64
cdd                          float64
dtype: object

In [8]:
# Display first 5 rows of output data
odf.head()

,block_id,transaction_hash,index,time,value,value_usd,recipient,type,script_hex,is_from_coinbase,is_spendable
0,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,0,2026-01-01 00:06:10,546,0.4778,3Eif1JfqeMERRsQHtvGEacNN9hhuvnsfe9,scripthash,a9148ee90177614ecde53314fd67c46162f315852a0787,1,-1
1,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,1,2026-01-01 00:06:10,314440653,275185.8750,3Awm3FNpmwrbvAFVThRUFqgpbVuqWisni9,scripthash,a9146582f2551e2a47e1ae8b03fb666401ed7c4552ef87,1,-1
2,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,1,2026-01-01 00:06:10,314440653,275185.8750,3Awm3FNpmwrbvAFVThRUFqgpbVuqWisni9,scripthash,a9146582f2551e2a47e1ae8b03fb666401ed7c4552ef87,1,-1
3,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,1,2026-01-01 00:06:10,314440653,275185.8750,3Awm3FNpmwrbvAFVThRUFqgpbVuqWisni9,scripthash,a9146582f2551e2a47e1ae8b03fb666401ed7c4552ef87,1,-1
4,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,2,2026-01-01 00:06:10,0,0.0000,d-bb48d5ef5dd5147a3c28f32e4055160c,nulldata,6a24aa21a9edd65a5527e7787b1a7d73682aed9e2d8d9d...,1,0


In [9]:
# Display data types of output dataframe
odf.dtypes

block_id              int64
transaction_hash     object
index                 int64
time                 object
value                 int64
value_usd           float64
recipient            object
type                 object
script_hex           object
is_from_coinbase      int64
is_spendable          int64
dtype: object

In [10]:
# Count occurrences of each recipient in input and output dataframes (address)
icounts = idf["recipient"].value_counts()
print(icounts)

ocounts = odf["recipient"].value_counts()
print(ocounts)

recipient
bc1pvad59hdlahwa66yv6qdn8vgd5d7p4usx88x54u0gqzz3y85yrcpq0fcfv2    46136
bc1pv6eqwxkxxda5nj49cry8z7lqnc2ft5hxz3jf0mx3v67e4c0krc8qzgjaa7    33957
bc1pwucn9l7vhfcp4k93xs5nu4r7tmd7qqmcnz5ekl877fp0l6cky7rqjgz4ck    29036
bc1pcecas3up00urhzjpqf5yagndfs7mhfmzuyl4yk68neqzgnsxjh9s7vk4ww    21451
bc1pxpyjtcmr8kuct4rtpfjjm4shc05zm08r0mzz5986tn5q5sulm58qq3zlam    19585
                                                                  ...  
13FnJDGha3aPD8e86T4CDhCyNf6KJ8jxuC                                    1
bc1qvf6jdkzcp27gytq57rxkqvhqanr8zgfmusn445                            1
bc1qr6l0wyx05qcm44gg9s9n66pz2ac76ftrnkk65q                            1
bc1qmkwdj2wcw6dkwa6e6dj957lpfze4f23lvck282                            1
bc1q74ex69ygl07fa7t9zm5xe09ym8e7uym5wy7gdmspqsr7fgryp7gsufpqhl        1
Name: count, Length: 400426, dtype: int64
recipient
d-39fb509d8eb10633e0e951c77bd3a7cb                                140748
bc1pvad59hdlahwa66yv6qdn8vgd5d7p4usx88x54u0gqzz3y85yrcpq0fcfv2     47860


In [11]:
# Merge input and output dataframes on transaction_hash and index
df = pd.merge(
    odf,
    idf,
    on=['transaction_hash','index'], # Merge on transaction_hash and index to link created (output) and spent (input) outputs
    how='left',
    suffixes=('_created','_spent') # Suffixes to distinguish created (output) and spent (input) columns
)

In [12]:
# Display first 5 rows of merged data
df.head()

,block_id_created,transaction_hash,index,time_created,value_created,value_usd_created,recipient_created,type_created,script_hex_created,is_from_coinbase_created,...,spending_block_id,spending_transaction_hash,spending_index,spending_time,spending_value_usd,spending_sequence,spending_signature_hex,spending_witness,lifespan,cdd
0,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,0,2026-01-01 00:06:10,546,0.4778,3Eif1JfqeMERRsQHtvGEacNN9hhuvnsfe9,scripthash,a9148ee90177614ecde53314fd67c46162f315852a0787,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,1,2026-01-01 00:06:10,314440653,275185.8750,3Awm3FNpmwrbvAFVThRUFqgpbVuqWisni9,scripthash,a9146582f2551e2a47e1ae8b03fb666401ed7c4552ef87,1,...,930442.0,c42bd5bb6c19218286f43589739190e541a6f850b146cb...,13.0,2026-01-01 15:32:47,275185.875,0.0,16001492858fe8411464453831172e18184f0b79a82924,304402200d8b7614b6db4c36164170b0654e75dfff78c9...,55597.0,2.023375
2,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,1,2026-01-01 00:06:10,314440653,275185.8750,3Awm3FNpmwrbvAFVThRUFqgpbVuqWisni9,scripthash,a9146582f2551e2a47e1ae8b03fb666401ed7c4552ef87,1,...,930442.0,c42bd5bb6c19218286f43589739190e541a6f850b146cb...,13.0,2026-01-01 15:32:47,275185.875,0.0,16001492858fe8411464453831172e18184f0b79a82924,304402200d8b7614b6db4c36164170b0654e75dfff78c9...,55597.0,2.023375
3,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,1,2026-01-01 00:06:10,314440653,275185.8750,3Awm3FNpmwrbvAFVThRUFqgpbVuqWisni9,scripthash,a9146582f2551e2a47e1ae8b03fb666401ed7c4552ef87,1,...,930442.0,c42bd5bb6c19218286f43589739190e541a6f850b146cb...,13.0,2026-01-01 15:32:47,275185.875,0.0,16001492858fe8411464453831172e18184f0b79a82924,304402200d8b7614b6db4c36164170b0654e75dfff78c9...,55597.0,2.023375
4,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,2,2026-01-01 00:06:10,0,0.0000,d-bb48d5ef5dd5147a3c28f32e4055160c,nulldata,6a24aa21a9edd65a5527e7787b1a7d73682aed9e2d8d9d...,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Get the number of rows in the merged dataframe
len(df)

1524592

In [14]:
# Remove rows with missing spending_transaction_hash (the UTXOs that have not been spent yet)
df = df.dropna(subset=['spending_transaction_hash'])
len(df)

766760

In [15]:
# Display columns of the merged dataframe
df.columns

Index(['block_id_created', 'transaction_hash', 'index', 'time_created',
       'value_created', 'value_usd_created', 'recipient_created',
       'type_created', 'script_hex_created', 'is_from_coinbase_created',
       'is_spendable_created', 'block_id_spent', 'time_spent', 'value_spent',
       'value_usd_spent', 'recipient_spent', 'type_spent', 'script_hex_spent',
       'is_from_coinbase_spent', 'is_spendable_spent', 'spending_block_id',
       'spending_transaction_hash', 'spending_index', 'spending_time',
       'spending_value_usd', 'spending_sequence', 'spending_signature_hex',
       'spending_witness', 'lifespan', 'cdd'],
      dtype='object')

In [16]:
# convert spending_time to datetime format
df['spending_time'] = pd.to_datetime(df['spending_time'])

In [28]:
class BarChartDetail:
    '''Class to hold details for bar chart creation, and used in BitcoinTran'''
    def __init__(self,x_col_name, y_col_name, xlabel, ylabel, title, path):
        self.x_col = x_col_name
        self.y_col = y_col_name
        self.xlabel = xlabel
        self.ylabel = ylabel
        self.title = title
        self.path = path
    
class BitcoinTran:
    '''Class to process bitcoin transaction data and create network graphs and statistics'''
    def __init__(self,st,df):
        self.st = st
        self.df = df
        self.df_window = None

    def set_df_window_mins(self,deltaMins): # set the dataframe for a time window of deltaMins minutes
        self.ed = self.st + pd.Timedelta(minutes=deltaMins)
        self.df_window = self.df[(self.df['spending_time'] >= self.st) & (self.df['spending_time'] < self.ed)]
        self.network_path = f'./Task3/network/{deltaMins}mins_network.gexf'
        print("number of rows for this window: ", len(self.df_window))
        return self

    def process_data(self,source,target,edge_attr): # create the networkx graph from a pandas dataframe
        self.G = nx.from_pandas_edgelist(
                self.df_window,
                source=source,
                target=target,
                edge_attr=edge_attr,
                create_using=nx.MultiDiGraph() # directed graph with multiple edges allowed
            )
        return self

    def create_stats_graph(self,sourceDetail,targetDetail,edgeDetail,volumeDetail): # create statistics graphs
        in_deg = pd.DataFrame(self.G.in_degree(), columns=['node', 'in_degree']) # in-degree dataframe
        out_deg = pd.DataFrame(self.G.out_degree(), columns=['node', 'out_degree']) # out-degree dataframe
        # Count multiple edges between same source and target
        edge_counts = Counter((u, v) for u, v, k in self.G.edges(keys=True))
        df_edges = pd.DataFrame([
                        {'source': u, 'target': v, 'counts': c}
                        for (u, v), c in edge_counts.items()
                    ])
        # Filter all with counts >= 2
        df_edges = df_edges[df_edges['counts'] >= 2]
        multi_target = in_deg[in_deg['in_degree'] >= 2]
        multi_source = out_deg[out_deg['out_degree'] >= 2]
        # Create bar charts for multi-source
        self.create_bar_chart(multi_source, sourceDetail.y_col,
                              sourceDetail.xlabel, sourceDetail.ylabel,
                              sourceDetail.title, sourceDetail.path)
        # Create bar charts for multi-target              
        self.create_bar_chart(multi_target, targetDetail.y_col,
                              targetDetail.xlabel, targetDetail.ylabel,
                              targetDetail.title, targetDetail.path)
        # Create bar charts for multi-edges
        self.create_bar_chart(df_edges, edgeDetail.y_col,
                              edgeDetail.xlabel, edgeDetail.ylabel,
                              edgeDetail.title, edgeDetail.path)
        # Create volume histogram
        self.create_volume_histogram(volumeDetail.x_col, 
                                     volumeDetail.xlabel, volumeDetail.ylabel,
                                     volumeDetail.title, volumeDetail.path)
        return self
    
    def create_network_graph(self): # save the networkx graph to a gexf file
        nx.write_gexf(self.G,self.network_path)    
        print(f"network file is created at {self.network_path}")
        return self

    def _create_bar_chart(self,counts_df,x,xlabel,ylabel,title,filename="./barchart"): # create a simple bar chart (old version)
        counts_df.to_csv(f"{filename}.csv",index=False)
        print(f"Raw dateframe is writtend to {filename}.csv")
        hist_df = counts_df[x].value_counts().reset_index()
        hist_df.columns = [x, 'number']
        n_bars = len(hist_df)
        width = max(8, n_bars * 0.7)
        height = 5 + math.log(n_bars + 1)
        limit = 300
        if max(hist_df['number']) <= limit:
            p = (
                   # ggplot(counts_df, aes(x='short_id', y=y,fill=x)) +
                ggplot(hist_df, aes(x=x, y='number')) +
                    geom_col() +
                    #labs(x=xlabel, y=ylabel, title=title, fill="Full hash") +
                labs(x=xlabel, y=ylabel, title=title) +
                  scale_y_log10() +
                    theme_bw() + 
                    theme(
                        figure_size=(width, height),
                        axis_text_x=element_text(rotation=0, ha='right'),
                        legend_text=element_text(size=8),
                        legend_title=element_text(size=9),
                        axis_title=element_text(size=11),
                        plot_title=element_text(size=13,hjust=0.5)
                    )
                )
        else:
            max_x = hist_df[x].max()
            p = (
                ggplot(hist_df, aes(x=x, y='number')) +
                    geom_col() +
                labs(x=xlabel, y=ylabel, title=title) +
                   scale_y_log10()  +
                    theme_bw() + 
                    theme(
                        figure_size=(width, height),
                        axis_text_x=element_text(rotation=0, ha='right'),
                        legend_text=element_text(size=8),
                        legend_title=element_text(size=9),
                        axis_title=element_text(size=11),
                        plot_title=element_text(size=13,hjust=0.5)
                    )
                )
        
        p.save(filename, dpi=300,limitsize=False)
        print(f"Bar chart saved to {filename}.png")
        
    def create_bar_chart(self, counts_df, x, xlabel, ylabel, title, filename="./barchart"): # create a bar chart with chunking for both samll and large datasets
        # Save the raw counts dataframe to csv
        counts_df.to_csv(f"{filename}.csv", index=False)
        print(f"Raw dataframe is written to {filename}.csv")

        # Aggregation & Sorting
        hist_df = counts_df[x].value_counts().reset_index()
        hist_df.columns = [x, 'number']
        hist_df = hist_df.sort_values(by=x)

        # Create Subplot Groups (Chunking)
        limit =100
        hist_df['subplot_group'] = [i // limit for i in range(len(hist_df))]
        #print(hist_df)
        # Create a nice label for the subplot headers (e.g., "Batch 1", "Batch 2")
        hist_df['group_label'] = "Batch " + (hist_df['subplot_group'] + 1).astype(str)
        n_subplots = hist_df['subplot_group'].max() + 1              # Total number of subplots needed
        
        # Calculate width based on the "fullest" subplot (usually 100)
        max_bars_in_subplot = min(len(hist_df), limit)
        width = max(10, max_bars_in_subplot * 0.15) 
        
        # Height needs to grow: Base height (4) * Number of subplots
        height = 4 * n_subplots 

        # Plotting
        p = (
            ggplot(hist_df, aes(x=x, y='number')) +
            geom_col(fill="steelblue") +
            labs(x=xlabel, y=ylabel, title=title) +
            scale_y_log10() + # Log scale for y-axis
            theme_bw() + 
            theme(
                figure_size=(width, height),
                axis_text_x=element_text(rotation=0, ha='center'),
                plot_title=element_text(size=13, hjust=0.5),
                axis_title=element_text(size=11),
                # Style the headers for each subplot
                strip_background=element_rect(fill="#e0e0e0"),
                strip_text=element_text(size=11, face="bold") 
            )
        )
        # More than 1 chunk => create subplots
        if n_subplots > 1:
            # scales='free_x': Each subplot gets its own independent x-axis & ncol=1: Stack them vertically
            p = p + facet_wrap('~group_label', scales='free_x', ncol=1)
        
        p.save(filename, dpi=300, limitsize=False)
        print(f"Bar chart saved to {filename}")

    def create_volume_histogram(self,amount_col,xlabel,ylabel,title,filename="./vol_hist"): # create volume histogram
        # Save transaction data to csv
        cols_to_save = ['transaction_hash', 'spending_transaction_hash']
        existing_cols = [c for c in cols_to_save if c in self.df_window.columns]
        if amount_col not in existing_cols and amount_col in self.df_window.columns:
            existing_cols.append(amount_col)
        self.df_window[existing_cols].sort_values(by=amount_col,ascending=False, kind="mergesort").to_csv(f"{filename}.csv", index=False)
        print(f"Transaction data saved to {filename}.csv")
        # Plotting
        p = (
            ggplot(self.df_window, aes(x=amount_col)) + 
            geom_histogram(bins=100, fill="orange", alpha=0.8) +
            scale_x_log10() +  scale_y_log10() +
            labs(x=xlabel, y=ylabel, title=title) +
            theme_bw() +
            theme(
                    axis_text_x=element_text(rotation=0, ha='center'),
                    legend_text=element_text(size=8),
                    legend_title=element_text(size=9),
                    axis_title=element_text(size=11),
                    plot_title=element_text(size=13,hjust=0.5)
                )
        )
        p.save(filename, dpi=300,limitsize=False)
        print(f"Volume histogram saved to {filename}.png")
        

In [29]:
# Set parameters
start_time = pd.to_datetime('2026-01-01 00:00:00')
source='transaction_hash'
target='spending_transaction_hash'
edge_attr='value_spent'
# Bar chart labels
xlabel_out = "Number of Targets from a Transaction hash"
xlabel_in = "Number of Sources from a Trasaction hash"
xlabel_par = "Number of repeated transaction path"

ylabel = "Counts - Log scale"

In [30]:
#  Declare BitcoinTran object
B = BitcoinTran(start_time,df)

In [31]:
# Titles for 10-minute windows
title_out = "Frequency of Multi-Target Transactions Between 00:00 to 00:10 on 2026-01-01"
title_in = "Frequency of Multi-Source Transactions Between 00:00 to 00:10 on 2026-01-01"
title_par = "Recurring Transaction Path Frequency Between 00:00 to 00:10 on 2026-01-01"
title_vol = "Distribution of Transaction Sizes \nBetween 00:00 to 00:10 on 2026-01-01"

# Create BitcoinTran object and process data for 10-minute window
source10 = BarChartDetail('node','out_degree',xlabel_out,ylabel,
                title_out,"./Task3/10mins/10mins_out")

target10 = BarChartDetail('node','in_degree',xlabel_in,ylabel,
                        title_in,"./Task3/10mins/10mins_in")

edge10 = BarChartDetail('source','counts',xlabel_par,ylabel,
                          title_par, "./Task3/10mins/10mins_par")

vol10 = BarChartDetail(edge_attr,'', 'Trans value - Log scale', 'Count - Log scale',
                       title_vol, "./Task3/10mins/10mins_vol")

In [32]:
# Create stats graphs for 10-minute window
b10 = B.set_df_window_mins(10).process_data(source,target,edge_attr).create_stats_graph(source10,target10,edge10,vol10)

number of rows for this window:  1906
Raw dataframe is written to ./Task3/10mins/10mins_out.csv
Bar chart saved to ./Task3/10mins/10mins_out
Raw dataframe is written to ./Task3/10mins/10mins_in.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 10 x 4 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/10mins/10mins_out
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 10 x 4 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/10mins/10mins_in
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Bar chart saved to ./Task3/10mins/10mins_in
Raw dataframe is written to ./Task3/10mins/10mins_par.csv
Bar chart saved to ./Task3/10mins/10mins_par
Transaction data saved to ./Task3/10mins/10mins_vol.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 10 x 4 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/10mins/10mins_par
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/10mins/10mins_vol
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Volume histogram saved to ./Task3/10mins/10mins_vol.png


In [33]:
# open the created file in Gephi
# Select ForceAtlas 2 as layout & tick Prevent Overlap and LinLog mode
# In Apperance:
#      Each node will be assigned a colour. A node will be more red if it has more sources of income.
#      Nodes tab:  
#             Select paint icon => Click Ranking => select In-Dgree => select rainbow => select invert
#      Each node has different size. A node will be bigger if it has more targets.
#      Nodes tab:
#             Select node_size icon => Click Ranking => select Out-Dgree => select min_size = 2 & max_size = 10
#      Each edge will be assigned a colour. An edge will be more red if it is higher value in $.
#      Edges tab:
#             Select paint icon => Click Ranking => select value_spent => select rainbow => select invert 
# Since nx.MultiDiGraph() is used, the parallel edges will be combined and thicken the edge between two nodes. 
b10.create_network_graph()

network file is created at ./Task3/network/10mins_network.gexf


In [34]:
# Plot the overall stats for all transaction on 2026-01-01
# Dont generate the gexf file, it wont be able to open by Gephi, too many nodes and edges to rendar.
title_out = "Frequency of Multi-Target Transactions Between 00:00 to 24:00 on 2026-01-01"
title_in = "Frequency of Multi-Source Transactions Between 00:00 to 24:00 on 2026-01-01"
title_par = "Recurring Transaction Path Frequency Between 00:00 to 24:00 on 2026-01-01"
title_vol = "Distribution of Transaction Sizes \nBetween 00:00 to 24:00 on 2026-01-01"

sourceday = BarChartDetail('node','out_degree',xlabel_out,ylabel,
                title_out,"./Task3/day/daymins_out")

targetday = BarChartDetail('node','in_degree',xlabel_in,ylabel,
                         title_in,"./Task3/day/daymins_in")

edgeday = BarChartDetail('source','counts',xlabel_par,ylabel,
                          title_par, "./Task3/day/daymins_par")

volday = BarChartDetail(edge_attr,'', 'Trans value - Log scale', ylabel,
                       title_vol, "./Task3/day/daymins_vol")

In [35]:
# generate the stats for daily, save it to ./Task3/day/
bday = B.set_df_window_mins(60*24).process_data(source,target,edge_attr).create_stats_graph(sourceday,targetday,edgeday,volday)

number of rows for this window:  766760
Raw dataframe is written to ./Task3/day/daymins_out.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 15.0 x 12 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/day/daymins_out
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Bar chart saved to ./Task3/day/daymins_out
Raw dataframe is written to ./Task3/day/daymins_in.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 15.0 x 16 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/day/daymins_in
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Bar chart saved to ./Task3/day/daymins_in
Raw dataframe is written to ./Task3/day/daymins_par.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 10 x 4 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/day/daymins_par
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Bar chart saved to ./Task3/day/daymins_par
Transaction data saved to ./Task3/day/daymins_vol.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/day/daymins_vol
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Volume histogram saved to ./Task3/day/daymins_vol.png


In [24]:
# Titles for 60-minute windows
title_out = "Frequency of Multi-Target Transactions Between 00:00 to 01:00 on 2026-01-01"
title_in = "Frequency of Multi-Source Transactions Between 00:00 to 01:00 on 2026-01-01"
title_par = "Recurring Transaction Path Frequency Between 00:00 to 01:00 on 2026-01-01"
title_vol = "Distribution of Transaction Sizes \nBetween 00:00 to 01:00 on 2026-01-01"

source60 = BarChartDetail('node','out_degree',xlabel_out,ylabel,
                title_out,"./60mins_out")

target60 = BarChartDetail('node','in_degree',xlabel_in,ylabel,
                         title_in,"./60mins_in")

edge60 = BarChartDetail('source','counts',xlabel_par,ylabel,
                          title_par, "./60mins_par")

vol60 = BarChartDetail(edge_attr,'', 'Trans value - Log scale', ylabel,
                       title_vol, "./60mins_vol")

In [25]:
# Create stats graphs for 60-minute window
b60 = B.set_df_window_mins(60).process_data(source,target,edge_attr).create_stats_graph(source60,target60,edge60,vol60)

number of rows for this window:  11737
Raw dataframe is written to ./60mins_out.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 10 x 4 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./60mins_out
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Bar chart saved to ./60mins_out
Raw dataframe is written to ./60mins_in.csv
Bar chart saved to ./60mins_in
Raw dataframe is written to ./60mins_par.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 10 x 4 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./60mins_in
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 10 x 4 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./60mins_par
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Bar chart saved to ./60mins_par
Transaction data saved to ./60mins_vol.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./60mins_vol
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Volume histogram saved to ./60mins_vol.png


In [ ]:
# Create network graph for 60-minute window
b60.create_network_graph()

network file is created at ./60mins_network.gexf
